In [1]:
import awkward as ak
import numpy as np
import time
# print(coffea.__version__)
from coffea import util
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
import mplhep as hep
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

from plugins import *
from triggerProcessor import *
from dask.distributed import Client

In [3]:
# in_year = 2018
# data_bool = True
# winterfell = True
# processor = triggerProcessor(year = in_year, trigger = 'AK8PFJet', data = data_bool)
# datastring = "JetHT" if processor.data == True else "QCDsim"
# if processor.data==False and winterfell:
#     filename = "QCD_flat.json"
# elif processor.data==False:
#     filename = "fileset_QCD.json"
# else:
#     filename = "datasets_UL_NANOAOD.json"
# result = runCoffeaJob(processor, jsonFile = filename, winterfell = winterfell, testing = False, year = processor.year, data = processor.data)
# with open('coffeaOutput/triggerAssignment_{}_{}_{}_NewHist.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
#     pickle.dump( result, f)
# # util.save(result, 'coffeaOutput/triggerAssignment_{}_{}_{}_NewHist.coffea'.format(datastring, processor.year, processor.trigger))

In [4]:
from scipy.optimize import minimize, curve_fit
from scipy.stats import rv_continuous
from scipy.special import erf, erfinv

def fit_function(x, b, c):
    # y = a+0.5*(1-a)*(1+erf((x-b)/c))
    y = 0.5*(1+erf((x-b)/(np.sqrt(2)*c)))
    return y
#### define custom error function using scipy.stats rv_continuous --> will automatically inherit MLE fit
class cust_erf(rv_continuous):
    def _pdf(self, x, b, c):
        # return a+0.5*(1-a)*(1+erf((x-b)/c))
        return 0.5*(1+erf((x-b)/(np.sqrt(2)*c)))
    #### use rv_continuous scale and loc to compute x->(x-loc)/scale
fit_func = cust_erf(name='erf')
print("rv_cont fit function: ", fit_func)

#### params are b,c
                      
                      
def fit_trigEff(x, data):
    #x is the pt bin centers; data is the efficiency values
    x0 = [np.min(x), np.min(x)/5]
    popt, pcov = curve_fit(fit_function, x, data, p0 = x0)
    b, c, loc, scale = fit_func.fit(data, *popt, floc = 0., fscale = 1.0)
    print("Basic fit results: ", popt)
    print("MLE results: ", [b, c, loc, scale])
    return b, c, popt

import matplotlib.pyplot as plt
plt.style.use([hep.style.CMS, hep.style.firamath])
import hist 
from plugins import checkdir
def plot_turnOn(result, HLT):
    hist_trigEff = result['hist_trigEff_ptCut']
    hist_trigRef = result['hist_trigRef']
    hist_pt = result['hist_pt']
    HLT_paths = [path for path in list(hist_trigEff.project("HLT_cat").axes[0])]
    dataset = hist_trigEff[{"HLT_cat":slice(0,hist.overflow-1,sum)}].axes[0][0]
    os_path = 'plots/triggerStudies'+ dataset + 'Testing/'
    checkdir(os_path)
    print('HLT_paths:', HLT_paths)
    trigThresh = [int(path.replace(HLT, '')) for path in HLT_paths]
    turnOnPts = {}
    turnOnPts_ptCut = {}
    for i in np.arange(len(HLT_paths)):
        path = HLT_paths[i]
        print("Path = ", path)
        print("dataset: ", dataset)
        # print("trigeffvalues for path: ", hist_trigEff[{"HLT_cat":path}])
        #### numerator is number of events that belong to trigger path below and have pt > current trigger thresh
        numerator = hist_trigEff[{"HLT_cat":path}].values()[0]
        #### denominator is number of events in trigger below
        denominator = hist_trigRef[{"HLT_cat":path}].values()[0]
        num = numerator[denominator > 0]
        denom = denominator[denominator > 0]
        efficiency = num / denom
        pt_centers = hist_trigEff.axes['pt'].centers[denominator > 0]
        
        print("Length of efficiency:", len(efficiency))
        #### plot original numerator and denominator histograms
        fig, ax = plt.subplots(1, 1)
        hist_trigEff[{"HLT_cat":path}].plot1d(ax=ax)
        hist_trigRef[{"HLT_cat":path}].plot1d(ax=ax)
        ax.set_title("Events in HLT " + HLT_paths[i-1])
        ax.set_xlabel("Leading Jet pT (GeV)")
        ax.set_ylim(1, None)
        ax.legend(["Numerator", "Numerator_ptCut", "Denominator"])
        plt.savefig(os_path + 'numDenom_HLT_' + path + ".png")
        #### Make custom fit function for the efficiencies
        if len(efficiency) > 0:
            fig, ax = plt.subplots(1, 1)
            ax.set_ylim([-0.01, 1.1])
            ### only fit points after trigger threshold
            eff_turnon_id = np.min(np.argwhere(trigThresh[i] <= pt_centers))
            b_mle, c_mle, popt = fit_trigEff(pt_centers[eff_turnon_id+3:-1], efficiency[eff_turnon_id+3:-1])
            # b_mle, c_mle, popt = fit_trigEff(pt_centers, efficiency)
            xpts = np.linspace(0, 2400, 2400)
            ### turn on point is pt value when efficiency reaches 0.99 --> solve fitted equation
            b, c = popt
            # to_pt = c*erfinv((2*0.99-1-a)/(1-a))+b
            to_pt = (np.sqrt(2)*c*erfinv(2*0.99-1)) + b
            to_pt_mle = (np.sqrt(2)*c_mle*erfinv(2*0.99-1)) + b_mle
            turnOnPts[path] = to_pt
            ax.scatter(hist_trigEff.axes['pt'].centers[denominator > 0], efficiency, label = 'HLT_' + path)
            ax.plot(xpts, fit_function(xpts, *popt), label = "Curve fit; to_pt = %.f"%to_pt)
            ax.plot(xpts, fit_function(xpts, b_mle, c_mle), label = "MLE fit; to_pt = %.f"%to_pt_mle)
            ax.set_xlabel("Leading Jet pT (GeV)")
            ax.legend()
            plt.savefig(os_path + 'efficiency_HLT_' + path + ".png")
    #### plot total leading jet pt for reference
    fig, ax = plt.subplots(1, 1)
#     hist_pt.plot1d(ax=ax)
    pt_hist = hist_trigEff[{"dataset":sum}]
    pt_hist.plot1d(ax=ax, overlay="HLT_cat", stack = True)
    ax.set_title("All events")
    ax.set_xlabel("Leading Jet pT (GeV)")
    ax.set_ylim(1, None)
    ax.set_xlim([180., 3200.])
    plt.legend()
    plt.savefig(os_path + 'HLT_' + path + ".png")
    
    return turnOnPts
    

rv_cont fit function:  <__main__.cust_erf object at 0x7f95a47dcfa0>


In [5]:
# HLT = 'AK8PFJet'
# with open("coffeaOutput/triggerAssignment_JetHT_2016_AK8PFJet_NewHist.pkl", "rb") as f:
#     result = pickle.load( f )
# turnOnPts = plot_turnOn(result, HLT)
# print(turnOnPts)

In [6]:
# print(turnOnPts)

In [7]:
# HLT = 'AK8PFJet'
# with open("coffeaOutput/triggerAssignment_QCDsim_2017_AK8PFJet_NewHist.pkl", "rb") as f:
#     result = pickle.load( f )
# turnOnPts = plot_turnOn(result, HLT)
# print(turnOnPts)

In [8]:
# HLT = 'AK8PFJet'
# result = util.load('coffeaOutput/triggerAssignment_QCDsim_2017_AK8PFJet_NewHist.coffea')
# turnOnPts = plot_turnOn(result, HLT)
# print(turnOnPts)

In [9]:
JetHT2016_new = {'AK8PFJet40':0.,
                 'AK8PFJet60': 0., 
                 'AK8PFJet80': 216.73271498356956, 
                 'AK8PFJet140': 288.77775439367343, 
                 'AK8PFJet200': 365.23692713436145, 
                 'AK8PFJet260': 447.2336225160111, 
                 'AK8PFJet320': 514.0097881035936, 
                 'AK8PFJet400': 632.9167920736127, 
                 'AK8PFJet450': 691.0929625368353, 
                 'AK8PFJet500': 750.4349697102186}

JetHT2017_new = {'AK8PFJet40':0.,
                 'AK8PFJet60': 0., 
                 'AK8PFJet80': 0., 
                 'AK8PFJet140': 280.13238125655073, 
                 'AK8PFJet200': 361.66204436543035, 
                 'AK8PFJet260': 448.6491079509249, 
                 'AK8PFJet320': 514.0894465691568, 
                 'AK8PFJet400': 630.6287526399205, 
                 'AK8PFJet450': 685.7915518743982, 
                 'AK8PFJet500': 741.8897640290605, 
                 'AK8PFJet550': 796.7956413028444}

JetHT2018_new = {'AK8PFJet15': 0.,
                 'AK8PFJet25': 0.,
                 'AK8PFJet40': 0.,
                 'AK8PFJet60': 0.,
                 'AK8PFJet80': 0., 
                 'AK8PFJet140': 281.81975103728195, 
                 'AK8PFJet200': 367.6531123322326, 
                 'AK8PFJet260': 458.9796186587356, 
                 'AK8PFJet320': 523.0660904166687, 
                 'AK8PFJet400': 638.7502963687824, 
                 'AK8PFJet450': 689.7766784490543, 
                 'AK8PFJet500': 745.1851326682686, 
                 'AK8PFJet550': 796.7749788024963}


turnOnPts_2016_JetHT = {'AK8PFJet40':0.,
                 'AK8PFJet60': 123.74869215024762, 
                 'AK8PFJet80': 186.56007840935342, 
                 'AK8PFJet140': 282.00447348652773, 
                 'AK8PFJet200': 314.92956425770416, 
                 'AK8PFJet260': 400.2994345896792, 
                 'AK8PFJet320': 454.7965847657287, 
                 'AK8PFJet400': 569.3376812308528, 
                 'AK8PFJet450': 625.8112613298829, 
                 'AK8PFJet500': 679.4291743756459}

turnOnPts_2017_JetHT = {'AK8PFJet40': 0.,
                 'AK8PFJet60': 75.97670877152461, 
                 'AK8PFJet80': 148.69545463038, 
                 'AK8PFJet140': 269.6308036095627, 
                 'AK8PFJet200': 319.9378872222969, 
                 'AK8PFJet260': 400.71346921684886, 
                 'AK8PFJet320': 463.5096627021751, 
                 'AK8PFJet400': 569.5345249797041, 
                 'AK8PFJet450': 627.0092235219273, 
                 'AK8PFJet500': 684.3538571622151, 
                 'AK8PFJet550': 741.7932656231794}

turnOnPts_2018_JetHT = {'AK8PFJet15': 0.,
                 'AK8PFJet25': 0.,
                 'AK8PFJet40': 0.,
                 'AK8PFJet60': 0.,
                 'AK8PFJet80': 198.47144674573286, 
                 'AK8PFJet140': 269.2033930314457, 
                 'AK8PFJet200': 318.57336482858017, 
                 'AK8PFJet260': 401.3103385970223, 
                 'AK8PFJet320': 461.85972273061157, 
                 'AK8PFJet400': 569.0542779590494, 
                 'AK8PFJet450': 626.5428359870361, 
                 'AK8PFJet500': 681.4582827805615, 
                 'AK8PFJet550': 737.606877311266}

turnOnPts_2016_QCDflat = {'AK8PFJet40':0.,
                          'AK8PFJet60': 106.79400044725446, 
                          'AK8PFJet80': 169.96352855878737, 
                          'AK8PFJet140': 265.2873364497926, 
                          'AK8PFJet200': 315.1712855987987, 
                          'AK8PFJet260': 385.9414264230047, 
                          'AK8PFJet320': 456.1785495557566, 
                          'AK8PFJet400': 528.4054384945691, 
                          'AK8PFJet450': 599.9567915817205, 
                          'AK8PFJet500': 671.8599581978826}

turnOnPts_2017_QCD_flat = {'AK8PFJet40':0.,
                           'AK8PFJet60': 78.72682467365811, 
                           'AK8PFJet80': 114.26008394028244, 
                           'AK8PFJet140': 262.69051582382326, 
                           'AK8PFJet200': 315.5082701033607, 
                           'AK8PFJet260': 391.2706193753585, 
                           'AK8PFJet320': 461.86703035451836, 
                           'AK8PFJet400': 545.2806371181599, 
                           'AK8PFJet450': 611.8168127002015, 
                           'AK8PFJet500': 681.8828608405956, 
                           'AK8PFJet550': 725.799090910656}

turnOnPts_2018_QCDflat = {'AK8PFJet15': 0.,
                          'AK8PFJet25': 0.,
                          'AK8PFJet40': 0.,
                          'AK8PFJet60': 0.,
                          'AK8PFJet80': 0.,
                          'AK8PFJet140': 255.02381002773194, 
                          'AK8PFJet200': 313.2026733979357, 
                          'AK8PFJet260': 388.2175317071178, 
                          'AK8PFJet320': 459.1304096129003, 
                          'AK8PFJet400': 530.7709332894112, 
                          'AK8PFJet450': 599.485074238053, 
                          'AK8PFJet500': 669.532328119605, 
                          'AK8PFJet550': 706.4936306567531}

turnOnPts_2016_QCDSim = {'AK8PFJet40':0.,
                         'AK8PFJet60': 0.,
                         'AK8PFJet80': 103.7431245681197,
                         'AK8PFJet140': 250.23583948077737,
                         'AK8PFJet200': 301.2418029415817,
                         'AK8PFJet260': 357.96787539254467,
                         'AK8PFJet320': 435.2061611936324,
                         'AK8PFJet400': 516.5390073904625,
                         'AK8PFJet450': 562.8044746691382,
                         'AK8PFJet500': 629.5738066119139}

turnOnPts_2017_QCDSim = {'AK8PFJet60': 0.,
                         'AK8PFJet80': 106.04213984155606,
                         'AK8PFJet140': 245.63662455880666,
                         'AK8PFJet200': 305.7948712092066,
                         'AK8PFJet260': 361.2655624387622,
                         'AK8PFJet320': 447.21717573414594,
                         'AK8PFJet400': 530.4549735674909,
                         'AK8PFJet450': 587.3825806338202,
                         'AK8PFJet500': 654.0716446127442,
                         'AK8PFJet550': 705.9374327651158 }



In [10]:
from plugins import *
from triggerProcessor import *
from dask.distributed import Client


in_year = 2016
data_bool = True
winterfell = True
processor = applyPrescales(year = in_year, trigger = 'AK8PFJet', turnOnPts = np.array(list(JetHT2016_new.values())), data = data_bool)
datastring = "JetHT" if processor.data == True else "QCDsim"
if processor.data==False and winterfell:
    filename = "QCD_flat.json"
elif processor.data==False:
    filename = "fileset_QCD.json"
else:
    filename = "datasets_UL_NANOAOD.json"
result = runCoffeaJob(processor, jsonFile = filename, winterfell = winterfell, testing = True, year = processor.year, data = processor.data)
with open('coffeaOutput/applyPrescales_{}_{}_{}_NewHist_test.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
    pickle.dump( result, f)

dataset =  /JetHT/Run2016F-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD
dataset =  /JetHT/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD
Running locally


Preprocessing:   0%|          | 0/2 [00:00<?, ?file/s]

Processing:   0%|          | 0/29 [00:00<?, ?chunk/s]

Event metadata:  {'dataset': '/JetHT/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD', 'filename': '/mnt/data/cms/store/data/Run2016H/JetHT/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/130000/0290F73B-A51C-A441-AEC1-8429F9CC8AA8.root', 'treename': 'Events', 'entrystart': 0, 'entrystop': 97985, 'fileuuid': 'bfc3e8a4-f011-11eb-9376-41cee183beef', 'version': 'latest'}
Event fields:  ['luminosityBlock', 'FsrPhoton', 'CaloMET', 'Muon', 'SoftActivityJetHT', 'SoftActivityJetNjets10', 'PV', 'DeepMETResolutionTune', 'FatJet', 'Flag', 'fixedGridRhoFastjetCentral', 'SoftActivityJetNjets2', 'SoftActivityJetNjets5', 'SubJet', 'MET', 'DeepMETResponseTune', 'boostedTau', 'fixedGridRhoFastjetCentralNeutral', 'SoftActivityJetHT10', 'fixedGridRhoFastjetCentralChargedPileUp', 'RawMET', 'TrigObj', 'RawPuppiMET', 'L1PreFiringWeight', 'Photon', 'ChsMET', 'Jet', 'CorrT1METJet', 'PuppiMET', 'SoftActivityJet', 'SoftActivityJetHT2', 'Tau', 'SV', 'Electron', 'Proton', 'HLTriggerFinalPath', 'fixedGridRhoFastjetCentralCal

/opt/conda/lib/python3.8/site-packages/coffea/processor/executor.py:261: UserWarning: Cancelling 3 running jobs (likely due to an exception)
  warnings.warn(


Event metadata:  {'dataset': '/JetHT/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD', 'filename': '/mnt/data/cms/store/data/Run2016H/JetHT/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/130000/0290F73B-A51C-A441-AEC1-8429F9CC8AA8.root', 'treename': 'Events', 'entrystart': 97985, 'entrystop': 195970, 'fileuuid': 'bfc3e8a4-f011-11eb-9376-41cee183beef', 'version': 'latest'}
Event fields:  ['luminosityBlock', 'FsrPhoton', 'CaloMET', 'Muon', 'SoftActivityJetHT', 'SoftActivityJetNjets10', 'PV', 'DeepMETResolutionTune', 'FatJet', 'Flag', 'fixedGridRhoFastjetCentral', 'SoftActivityJetNjets2', 'SoftActivityJetNjets5', 'SubJet', 'MET', 'DeepMETResponseTune', 'boostedTau', 'fixedGridRhoFastjetCentralNeutral', 'SoftActivityJetHT10', 'fixedGridRhoFastjetCentralChargedPileUp', 'RawMET', 'TrigObj', 'RawPuppiMET', 'L1PreFiringWeight', 'Photon', 'ChsMET', 'Jet', 'CorrT1METJet', 'PuppiMET', 'SoftActivityJet', 'SoftActivityJetHT2', 'Tau', 'SV', 'Electron', 'Proton', 'HLTriggerFinalPath', 'fixedGridRhoFastjetCentr

ValueError: cannot broadcast RegularArray of size 71100 with RegularArray of size 82011

(https://github.com/scikit-hep/awkward-1.0/blob/1.7.0/src/awkward/_util.py#L936)

Failed processing file: WorkItem(dataset='/JetHT/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD', filename='/mnt/data/cms/store/data/Run2016H/JetHT/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/130000/0290F73B-A51C-A441-AEC1-8429F9CC8AA8.root', treename='Events', entrystart=0, entrystop=97985, fileuuid=b'\xbf\xc3\xe8\xa4\xf0\x11\x11\xeb\x93vA\xce\xe1\x83\xbe\xef', usermeta={})

In [ ]:
trigger = 'AK8PFJet'

#are these prescales for JetHT data only? are they also valid for QCD flat and binned MC?
#### need to run brilCalc for each run -- make double nested dict JSON file


trigThresh_2016 = [40, 60, 80, 140, 200, 260, 320, 400, 450, 500]
HLT_paths_2016 = [trigger + str(i) for i in trigThresh_2016]
prescales_2016 = {'AK8PFJet40'  : 136006.59,
                 'AK8PFJet60'  : 50007.75,
                 'AK8PFJet80'  : 13163.18,
                 'AK8PFJet140' : 1501.12,
                 'AK8PFJet200' : 349.82,
                 'AK8PFJet260' : 61.17,
                 'AK8PFJet320' : 20.49,
                 'AK8PFJet400' : 6.99,
                 'AK8PFJet450' : 1.00,
                 'AK8PFJet500' : 1.00 }
tot_lumi_2016 = 36.31

trigThresh_2017 = [40, 60, 80, 140, 200, 260, 320, 400, 450, 500, 550]
HLT_paths_2017 = [trigger + str(i) for i in trigThresh_2017]
prescales_2017 = {'AK8PFJet40'  : 86061.17, 
                 'AK8PFJet60'  : 36420.75,    
                 'AK8PFJet80'  : 9621.74,    
                 'AK8PFJet140' : 1040.40,  
                 'AK8PFJet200' : 189.54,      
                 'AK8PFJet260' : 74.73,     
                 'AK8PFJet320' : 29.49,      
                 'AK8PFJet400' : 9.85,       
                 'AK8PFJet450' : 3.97,       
                 'AK8PFJet500' : 1.00,       
                 'AK8PFJet550' : 1.00 }
tot_lumi_2018 = 41.48

trigThresh_2018 = [15, 25, 40, 60, 80, 140, 200, 260, 320, 400, 450, 500, 550]
HLT_paths_2018 = [trigger + str(i) for i in trigThresh_2018]
prescales_2018 = {'AK8PFJet15'  : 318346231.66,
                 'AK8PFJet25'  : 318346231.66,
                 'AK8PFJet40'  : 248642.75,
                 'AK8PFJet60'  : 74330.16,  
                 'AK8PFJet80'  : 11616.52,   
                 'AK8PFJet140' : 1231.88, 
                 'AK8PFJet200' : 286.14,    
                 'AK8PFJet260' : 125.78,   
                 'AK8PFJet320' : 32.66,   
                 'AK8PFJet400' : 15.83,      
                 'AK8PFJet450' : 7.96,      
                 'AK8PFJet500' : 1.00,  
                 'AK8PFJet550' : 1.00 }
tot_lumi_2018 = 59.83

In [ ]:
def plotPrescales(result, year, prescales):
    hist_pt = result['hist_pt'][{'dataset':sum}]
    hist_pt_byHLTpath = result['hist_pt_byHLTpath'][{'dataset':sum}]
    plt.rcParams['figure.facecolor']='white'
    plt.rcParams["figure.figsize"] = (30,20)
    fig, axs = plt.subplots(2, 2)      
    print(hist_pt)
    hist_pt.plot1d(ax = axs[0,0], overlay='HLT_cat')
    axs[0,0].set_yscale('log')
    axs[0,0].set_ylim([0.0, 1.0e9])
    axs[0,0].set_title('Events sorted by HLT')
    
    for i, name in enumerate(hist_pt.axes["HLT_cat"]):
        hist_pt.view(flow=True)[i] *= prescales[name]

    hist_pt.plot1d(ax = axs[0,1], overlay='HLT_cat')
    axs[0,1].set_yscale('log')
    axs[0,1].set_ylim([0.0, 1.0e10])
    axs[0,1].set_title('Events sorted by HLT w/ prescale applied')

    hist_pt_byHLTpath.plot1d(ax = axs[1,0], overlay='HLT_cat')
    axs[1,0].set_yscale('log')
    axs[1,0].set_ylim([0.0, 1.0e9])
    axs[1,0].set_title('Doubles removed')
    
    for i, name in enumerate(hist_pt_byHLTpath.axes["HLT_cat"]):
        hist_pt_byHLTpath.view(flow=True)[i] *= prescales[name]

    hist_pt_byHLTpath.plot1d(ax = axs[1,1], overlay='HLT_cat')
    axs[1,1].set_yscale('log')
    axs[1,1].set_ylim([1.0e-1, 1.0e10])
    axs[1,1].set_title('Doubles removed w/ prescale applied')

    fig.suptitle('Apply Prescales ' + str(year), fontsize="large")

    plt.rc('legend',fontsize='small')
    plt.legend()
    dataset = 'JetHT' + str(year)
    os_path = 'plots/triggerStudies'+ dataset + '/'
    checkdir(os_path)
    plt.savefig(os_path + 'applyPrescales' + str(year) + ".png")


In [ ]:
with open("coffeaOutput/applyPrescales_JetHT_2016_AK8PFJet_NewHist.pkl", "rb") as f:
    result = pickle.load( f )
# print(result['hist_pt'][{'HLT_cat':sum, 'pt':sum}].values())
plotPrescales(result, 2016, prescales_2016)

In [ ]:
with open("coffeaOutput/applyPrescales_JetHT_2017_AK8PFJet_NewHist.pkl", "rb") as f:
    result = pickle.load( f )
# print(result['hist_pt'][{'HLT_cat':sum, 'pt':sum}].values())
plotPrescales(result, 2017, prescales_2017)

In [ ]:
with open("coffeaOutput/applyPrescales_JetHT_2018_AK8PFJet_NewHist.pkl", "rb") as f:
    result = pickle.load( f )
# print(result['hist_pt'][{'HLT_cat':sum, 'pt':sum}].values())
plotPrescales(result, 2018, prescales_2018)